In [ ]:
# trying to incorporate some ml stuff to this nba data now, too

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
lebron = pd.read_csv('data/NBA/1_lebron_james_shot_chart_1_2023.csv')
james = pd.read_csv('data/NBA/2_james_harden_shot_chart_2023.csv')
stephen = pd.read_csv('data/NBA/3_stephen_curry_shot_chart_2023.csv')

print(lebron.shape)
print(james.shape)
print(stephen.shape)

In [ ]:
lebron.dtypes

In [ ]:
# let's try to use the positional data from 'top' and 'left' to create a 
# model that predicts whether lebron james will make a shot or not

# first let's select the top and left columns for X_train
# and shots made for the y_train

X_train = lebron.iloc[0:1300,0:2].to_numpy().astype('int16')
y_train = lebron.iloc[0:1300,5].to_numpy().astype('int8')

X_valid = lebron.iloc[1300:1533,0:2].to_numpy().astype('int16')
y_valid = lebron.iloc[1300:1533,5].to_numpy().astype('int8')


In [ ]:
X_train_adder = lebron.iloc[0:1300,7].to_numpy().astype('int16')
X_train = np.concatenate((X_train, X_train_adder.reshape(-1,1)), axis=1)
X_train

X_valid_adder = lebron.iloc[1300:1533,7].to_numpy().astype('int16')
X_valid = np.concatenate((X_valid, X_valid_adder.reshape(-1,1)), axis=1)
X_valid

In [ ]:
X_train

In [ ]:
# now let's scale the data

# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()

# X_train = scaler.fit_transform(X_train)
# X_train

In [ ]:
X_train.shape

In [ ]:
# time to create a model
from tensorflow.keras import initializers
from tensorflow.keras import layers

tf.keras.backend.clear_session()

# creating initialization variable
init = tf.keras.initializers.RandomNormal(mean=0., stddev=1., seed=None)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(3, activation='relu', input_shape=[3]),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, 
                    y_train,
                    epochs=50,
                    validation_data=(X_valid, y_valid))

In [ ]:
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot()

In [ ]:
# look into AlexNet in regards to MNIST
# builing different models
# write a python script that saves the network at it's maximum accuracy
# write a python script that stops training when the network stops improving
# also save the state of the model at it's maximum accuracy
# look into ResNet in regards to MNIST

# try to achieve 99% accuracy on MNIST without changing the network architecture

# look into flattening

# look into softmax

# get MNIST to 99% accuracy

# spanning pooling pyramids

# convulational layer
# frozen layer
# dense layer
# softmax layer

# what's a memory leak?

# refresher on CS101 stuff